In [119]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.ensemble import RandomForestRegressor
from sklearn import cross_validation
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor

In [92]:
sales = pd.read_csv("../data/train.csv")
def transformHoliday(x):
    if x == "a":
        return 1
    if x == "b":
        return 2
    if x == "c":
        return 3
    return 0
sales['StateHoliday'] = sales['StateHoliday'].apply(transformHoliday)

In [93]:
sales = sales.drop(['Date'], axis=1)

In [107]:
X, y = sales.drop(['Sales'], axis=1), sales['Sales']

In [110]:
names = X.columns 

In [111]:
X.sample(3)

,Store,DayOfWeek,Customers,Open,Promo,StateHoliday,SchoolHoliday
916283,539,1,0,0,0,2,1
75846,27,7,0,0,0,0,0
664458,704,3,982,1,0,0,0


In [112]:
y.sample(3)

375654    4557
951432    5545
239930       0
Name: Sales, dtype: int64

In [113]:
X, y = X.values, y.values
X, X_test, y, y_test = cross_validation.train_test_split(X, y, test_size=0.1)

In [114]:
randomforest = RandomForestRegressor(n_estimators=10, max_depth=10, min_samples_split=5, n_jobs=-1, verbose=5)

In [115]:
randomforest.fit(X, y)

[Parallel(n_jobs=-1)]: Done   1 out of  10 | elapsed:    3.9s remaining:   34.9s
[Parallel(n_jobs=-1)]: Done   3 out of  10 | elapsed:    4.0s remaining:    9.2s
[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed:    8.3s remaining:    5.5s
[Parallel(n_jobs=-1)]: Done   9 out of  10 | elapsed:   10.5s remaining:    1.2s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   10.5s finished


building tree 1 of 10
building tree 2 of 10
building tree 3 of 10
building tree 4 of 10
building tree 5 of 10
building tree 6 of 10
building tree 7 of 10
building tree 8 of 10
building tree 9 of 10
building tree 10 of 10


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=10,
           max_features='auto', max_leaf_nodes=None, min_samples_leaf=1,
           min_samples_split=5, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=-1, oob_score=False, random_state=None,
           verbose=5, warm_start=False)

In [116]:
randomforest.score(X_test, y_test)

[Parallel(n_jobs=4)]: Done   1 out of  10 | elapsed:    0.0s remaining:    0.2s
[Parallel(n_jobs=4)]: Done   3 out of  10 | elapsed:    0.0s remaining:    0.1s
[Parallel(n_jobs=4)]: Done   6 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   9 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    0.0s finished


0.8933592904933314

In [117]:
print sorted(zip(map(lambda x: round(x, 4), randomforest.feature_importances_), names), 
             reverse=True)

[(0.9487, 'Customers'), (0.0263, 'Promo'), (0.0164, 'Store'), (0.0084, 'DayOfWeek'), (0.0002, 'SchoolHoliday'), (0.0, 'StateHoliday'), (0.0, 'Open')]


In [118]:
# note the variable 'Open' probably have 0 importance because the # of Customers already encapsulates the 'Open' information

In [126]:
adaboost = AdaBoostRegressor(DecisionTreeRegressor(max_depth=15), n_estimators=50, loss='square', learning_rate=1)

In [127]:
adaboost.fit(X, y)

AdaBoostRegressor(base_estimator=DecisionTreeRegressor(criterion='mse', max_depth=15, max_features=None,
           max_leaf_nodes=None, min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, random_state=None,
           splitter='best'),
         learning_rate=1, loss='square', n_estimators=50,
         random_state=None)

In [128]:
adaboost.score(X_test, y_test)

0.92250154587419497

In [129]:
print sorted(zip(map(lambda x: round(x, 4), adaboost.feature_importances_), names), 
             reverse=True)

[(0.7326, 'Customers'), (0.2298, 'Store'), (0.0176, 'DayOfWeek'), (0.0164, 'Promo'), (0.0034, 'SchoolHoliday'), (0.0002, 'StateHoliday'), (0.0, 'Open')]


In [ ]:
# WTF why is adagrad worse than RF in sklearn...